In [2]:
import matplotlib.pyplot as plt
from itertools import combinations
from math import exp
from scipy.stats import norm
from statsmodels.formula.api import ols
from scipy import stats
import seaborn as sns
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import warnings
import pickle

file = pd.ExcelFile('NASSAgcensusDownload2017.xlsx')
CandP = pd.read_excel(file, 'Crops and Plants')
SandI = pd.read_excel(file, 'Economics')

In [3]:
df = pd.concat([CandP, SandI], 1)
df.head()

,FIPS,FIPSTEXT,y17_M123_valueText,y17_M123_classRange,y17_M123_valueNumeric,y17_M124_valueText,y17_M124_classRange,y17_M124_valueNumeric,y17_M125_valueText,y17_M125_classRange,...,y17_M073_valueNumeric,y17_M074_valueText,y17_M074_classRange,y17_M074_valueNumeric,y17_M075_valueText,y17_M075_classRange,y17_M075_valueNumeric,y17_M202_valueText,y17_M202_classRange,y17_M202_valueNumeric
0,0,0,14.60,NaN,14.60,26.48,NaN,26.48,-1.63,NaN,...,133363.0,79,NaN,79.0,661,NaN,661.0,400603,NaN,400603.0
1,1001,1001,(D),Less than 5,NaN,2.34,Less than 5,2.34,0.43,Sparse Data,...,96682.0,41,25 - 49,41.0,104,Less than 200,104.0,0,"Less than 50,000",0.0
2,1003,1003,3.12,Less than 5,3.12,7.06,5 - 14,7.06,0.94,Sparse Data,...,136496.0,53,50 - 74,53.0,216,200 - 399,216.0,0,"Less than 50,000",0.0
3,1005,1005,(D),5 - 19,NaN,4.22,Less than 5,4.22,-0.35,Sparse Data,...,87026.0,28,25 - 49,28.0,161,Less than 200,161.0,(D),"Less than 50,000",NaN
4,1007,1007,0.00,Less than 5,0.00,7.65,5 - 14,7.65,(D),Sparse Data,...,68656.0,29,25 - 49,29.0,(D),200 - 399,NaN,0,"Less than 50,000",0.0


In [6]:
test_list = [df['y17_M123_valueText'], df['y17_M123_classRange'], df['y17_M123_valueNumeric']]
tested_df = pd.DataFrame(test_list).T
tested_df

,y17_M123_valueText,y17_M123_classRange,y17_M123_valueNumeric
0,14.60,NaN,14.6
1,(D),Less than 5,NaN
2,3.12,Less than 5,3.12
3,(D),5 - 19,NaN
4,0.00,Less than 5,0
...,...,...,...
3075,0.00,Less than 5,0
3076,0.00,Less than 5,0
3077,0.00,Less than 5,0
3078,63.85,40 - 69,63.85


In [33]:
feature = []
for i in df.columns:
    if 'valueNumeric' not in i:
        feature.append(i)
df = df.drop(columns=feature)

In [93]:
CandP.shape

(3080, 236)

In [94]:
CandP.dropna().shape

(13, 236)

In [101]:
a=list(df['y17_M041_valueNumeric'].dropna(axis=0))
a

[21478.0,
 20586.0,
 13758.0,
 12182.0,
 14584.0,
 7065.0,
 17048.0,
 8983.0,
 6937.0,
 19706.0,
 9410.0,
 10208.0,
 23719.0,
 12894.0,
 7786.0,
 11683.0,
 9311.0,
 6891.0,
 21147.0,
 6789.0,
 6239.0,
 7275.0,
 7100.0,
 10590.0,
 22022.0,
 6621.0,
 8567.0,
 9678.0,
 6269.0,
 13204.0,
 8902.0,
 10687.0,
 11617.0,
 14931.0,
 18090.0,
 8232.0,
 7485.0,
 4407.0,
 9204.0,
 7038.0,
 4474.0,
 16859.0,
 8231.0,
 9435.0,
 11040.0,
 7739.0,
 12832.0,
 5365.0,
 10545.0,
 19132.0,
 13348.0,
 14050.0,
 4522.0,
 13916.0,
 22767.0,
 9297.0,
 11696.0,
 24347.0,
 7562.0,
 15963.0,
 34197.0,
 7782.0,
 13404.0,
 7022.0,
 7437.0,
 29803.0,
 26302.0,
 5923.0,
 20436.0,
 56256.0,
 11870.0,
 24548.0,
 9751.0,
 16371.0,
 21100.0,
 33262.0,
 41567.0,
 14581.0,
 72741.0,
 108785.0,
 84002.0,
 37255.0,
 13841.0,
 60049.0,
 78839.0,
 13018.0,
 20140.0,
 88988.0,
 30649.0,
 38902.0,
 9716.0,
 8209.0,
 4382.0,
 13523.0,
 8144.0,
 7746.0,
 29248.0,
 15328.0,
 17246.0,
 5063.0,
 24096.0,
 9706.0,
 10437.0,
 32044.0,


In [9]:
df['y17_M041_valueNumeric'].name

'y17_M041_valueNumeric'

In [102]:
b=df[df['y17_M041_valueNumeric']==a]
b

ValueError: Lengths must match to compare

In [85]:
df_dropped = df['y17_M041_valueNumeric'].dropna()
df_dropped

0        21478.0
1        20586.0
2        13758.0
3        12182.0
4        14584.0
          ...   
3075     27047.0
3076    121272.0
3077     15299.0
3078     25735.0
3079     15223.0
Name: y17_M041_valueNumeric, Length: 3037, dtype: float64

In [34]:
x_train, x_test, y_train, y_test = train_test_split(features, df_dropped)

ValueError: At least one array required as input

In [78]:
def groupIDs(df):
    all_parts = []
    for i in df['MapID']:
        parts = []
        for x in df.columns:
            if (i in x): #column contains mapid string
                parts.append(x)
        all_parts.append(picker(pd.DataFrame(parts))) #chosen filled column returned as series
    return pd.DataFrame(all_parts).T


#takes a df of columns to compare data
#returns a single column with the best value from the columns
def picker(df):
    if (df.shape[0] >= 4):
        df = df.T
    for c in df.column :
        best_val = None
        for i in c:
            if (type(i) == int or type(i) == float):
                best_val = i
                break;
                
def turnintoint(z): # element 
    if z.isalpha(): #case where entire x is str
        return np.nan
    elif ('less than' or 'greater than' in z):
        parts = z.split(' ')
        return int(z[-1]) - 1
    elif ('-' in z):
        parts = z.split(" ")
        return (int(parts[0]) + int(parts[-1])) // 2
    else :
        return float(z)

In [66]:
df.columns

Index(['FIPS', 'FIPSTEXT', 'y17_M123_valueText', 'y17_M123_classRange',
       'y17_M123_valueNumeric', 'y17_M124_valueText', 'y17_M124_classRange',
       'y17_M124_valueNumeric', 'y17_M125_valueText', 'y17_M125_classRange',
       ...
       'y17_M073_valueNumeric', 'y17_M074_valueText', 'y17_M074_classRange',
       'y17_M074_valueNumeric', 'y17_M075_valueText', 'y17_M075_classRange',
       'y17_M075_valueNumeric', 'y17_M202_valueText', 'y17_M202_classRange',
       'y17_M202_valueNumeric'],
      dtype='object', length=388)

In [67]:
df.y17_M123_classRange.unique() #apply(lambda x: turnintoint(x))

array([nan, 'Less than 5', '5 - 19', '20 - 39', '40 - 69', '90 or more',
       '70 - 89'], dtype=object)

In [79]:
df.y17_M123_classRange.apply(lambda x: turnintoint(str(x)))

ValueError: invalid literal for int() with base 10: 'e'

In [62]:
list1=[]
for c in df.columns:
    #append((lambda x: turnintoint(x),list(df[c]))))
    df[c] = df[c].apply(lambda x: turnintoint(x))

AttributeError: ("'Series' object has no attribute 'isalpha'", 'occurred at index FIPS')

In [12]:

a = ['str', 'str1', 'str2']

SyntaxError: invalid syntax (<ipython-input-12-b2f5e9ade7b4>, line 2)

In [52]:
b = ['123', '23']
b[-1].isalpha()

#b.isalpha() #returns false if number in 

False

In [59]:
'osegnq29u83y59'.isalpha()

False

In [55]:
# df[].sum()